In [1]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import pandas as pd

df = pd.read_excel("data2.xlsx", header=0)
df["Date"] = pd.to_datetime(df["Date"])
df

,Unnamed: 0,Date,Contest number,Word,Number of reported results,Number in hard mode,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X),ratio,vowel,len,uniq,star
0,0,2022-01-07,202,slump,80630,1362,1,3,23,39,24,9,1,0.016892,1,5,5,NaN
1,1,2022-01-08,203,crank,101503,1763,1,5,23,31,24,14,2,0.017369,1,5,5,NaN
2,2,2022-01-09,204,gorge,91477,1913,1,3,13,27,30,22,4,0.020912,2,5,4,NaN
3,3,2022-01-10,205,query,107134,2242,1,4,16,30,30,17,2,0.020927,2,5,5,NaN
4,4,2022-01-11,206,drink,153880,3017,1,9,35,34,16,5,1,0.019606,1,5,5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,2022-12-27,556,condo,20879,2012,0,2,17,35,29,14,3,0.096365,2,5,4,NaN
351,351,2022-12-28,557,impel,20160,1937,0,3,21,40,25,9,1,0.096081,2,5,5,NaN
352,352,2022-12-29,558,havoc,20001,1919,0,2,16,38,30,12,2,0.095945,2,5,5,NaN
353,353,2022-12-30,559,molar,21204,1973,0,4,21,38,26,9,1,0.093048,2,5,5,NaN


In [2]:
df = df.drop(df[df["len"] == 4].index.values)
df = df.drop(df[df["len"] == 6].index.values)
len(df)

355

In [3]:
dic = {j: i
       for i, j in enumerate('abcdefghijklmnopqrstuvwxyz', 0)}
dic["ï"] = dic["i"]
dic

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19,
 'u': 20,
 'v': 21,
 'w': 22,
 'x': 23,
 'y': 24,
 'z': 25,
 'ï': 8}

In [4]:
words_list = []
for i in df["Word"].values:
    words_list.append(([dic[j] for j in i]))
# words_list = torch.Tensor(words_list)
words_list

[[18, 11, 20, 12, 15],
 [2, 17, 0, 13, 10],
 [6, 14, 17, 6, 4],
 [16, 20, 4, 17, 24],
 [3, 17, 8, 13, 10],
 [0, 1, 1, 4, 24],
 [19, 0, 13, 6, 24],
 [15, 0, 13, 8, 2],
 [18, 14, 11, 0, 17],
 [18, 7, 8, 17, 4],
 [15, 17, 14, 23, 24],
 [15, 14, 8, 13, 19],
 [17, 14, 1, 14, 19],
 [15, 17, 8, 2, 10],
 [22, 8, 13, 2, 4],
 [2, 17, 8, 12, 15],
 [10, 13, 14, 11, 11],
 [18, 20, 6, 0, 17],
 [22, 7, 0, 2, 10],
 [12, 14, 20, 13, 19],
 [15, 4, 17, 10, 24],
 [2, 14, 20, 11, 3],
 [22, 17, 20, 13, 6],
 [11, 8, 6, 7, 19],
 [19, 7, 14, 18, 4],
 [12, 14, 8, 18, 19],
 [18, 7, 0, 17, 3],
 [15, 11, 4, 0, 19],
 [0, 11, 14, 5, 19],
 [18, 10, 8, 11, 11],
 [4, 11, 3, 4, 17],
 [5, 17, 0, 12, 4],
 [7, 20, 12, 14, 17],
 [15, 0, 20, 18, 4],
 [20, 11, 2, 4, 17],
 [20, 11, 19, 17, 0],
 [17, 14, 1, 8, 13],
 [2, 24, 13, 8, 2],
 [0, 17, 14, 12, 0],
 [2, 0, 20, 11, 10],
 [18, 7, 0, 10, 4],
 [3, 14, 3, 6, 4],
 [18, 22, 8, 11, 11],
 [19, 0, 2, 8, 19],
 [14, 19, 7, 4, 17],
 [19, 7, 14, 17, 13],
 [19, 17, 14, 21, 4],
 [1, 11,

In [5]:
per = df.iloc[:, 6: 13]
per = [([j / 100 for j in i]) for i in per.values]
per

[[0.01, 0.03, 0.23, 0.39, 0.24, 0.09, 0.01],
 [0.01, 0.05, 0.23, 0.31, 0.24, 0.14, 0.02],
 [0.01, 0.03, 0.13, 0.27, 0.3, 0.22, 0.04],
 [0.01, 0.04, 0.16, 0.3, 0.3, 0.17, 0.02],
 [0.01, 0.09, 0.35, 0.34, 0.16, 0.05, 0.01],
 [0.01, 0.02, 0.13, 0.29, 0.31, 0.2, 0.03],
 [0.01, 0.04, 0.21, 0.3, 0.24, 0.15, 0.05],
 [0.01, 0.09, 0.35, 0.34, 0.16, 0.05, 0.01],
 [0.01, 0.09, 0.32, 0.32, 0.18, 0.07, 0.01],
 [0.01, 0.08, 0.32, 0.32, 0.18, 0.08, 0.02],
 [0.01, 0.02, 0.11, 0.24, 0.31, 0.26, 0.06],
 [0.01, 0.16, 0.37, 0.28, 0.12, 0.04, 0.01],
 [0.01, 0.08, 0.29, 0.34, 0.2, 0.08, 0.01],
 [0.01, 0.08, 0.3, 0.33, 0.19, 0.07, 0.01],
 [0.01, 0.03, 0.17, 0.33, 0.29, 0.15, 0.03],
 [0.01, 0.05, 0.28, 0.38, 0.2, 0.07, 0.01],
 [0.01, 0.01, 0.11, 0.29, 0.33, 0.21, 0.04],
 [0.01, 0.06, 0.25, 0.34, 0.23, 0.09, 0.01],
 [0.01, 0.04, 0.22, 0.37, 0.24, 0.1, 0.02],
 [0.01, 0.09, 0.29, 0.33, 0.19, 0.07, 0.01],
 [0.01, 0.04, 0.17, 0.3, 0.27, 0.17, 0.04],
 [0.01, 0.07, 0.29, 0.35, 0.2, 0.08, 0.01],
 [0.0, 0.02, 0.18, 0.

In [6]:
x_train, x_test, y_train, y_test = train_test_split(words_list, per, test_size=0.1, random_state=50)
len(x_train), len(x_test), len(y_train), len(y_test)

(319, 36, 319, 36)

In [7]:
x_train = torch.LongTensor(x_train)
x_test = torch.LongTensor(x_test)
y_train = torch.Tensor(y_train)
y_test = torch.Tensor(y_test)
x_test.shape

torch.Size([36, 5])

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [9]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.embedding = nn.Embedding(26, 5),
        self.net = nn.Sequential(
            nn.Linear(5, 30),
            nn.ReLU(),
            nn.Linear(30, 7),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.net(x)
        # out = [[j / sum(i) for j in i] for i in out.tolist()]
        s = torch.sum(out, axis=1)
        s.unsqueeze_(0)
        s = s.repeat_interleave(7, axis=1).view(-1, 7)
        return torch.div(out, s)

In [10]:
model = MLP().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [11]:
embedding = nn.Embedding(26, 1)
x_train = embedding(x_train)
x_train = x_train.view(-1, 5)
x_train

tensor([[ 0.4141, -0.8780,  0.3245,  0.1524, -1.3400],
        [ 0.1524, -0.8780, -1.8069, -1.6953,  0.2413],
        [-0.5813, -0.0361,  1.2400, -2.4450,  0.3922],
        ...,
        [ 0.7993, -0.6880,  0.4141,  0.4141,  0.2413],
        [-1.6953, -1.3400, -0.6880, -0.1886, -2.4450],
        [-1.6953,  0.5624,  0.1547,  0.5624,  0.3245]],
       grad_fn=<ViewBackward0>)

In [12]:
x_test = embedding(x_test)
x_test = x_test.view(-1, 5)
x_test.shape

torch.Size([36, 5])

In [13]:
x_train = x_train.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

In [14]:
import numpy as np


def train(num_epochs):
    pred = ""
    for epoch in range(num_epochs):
        pred = model(x_train)
        # print(pred.tolist())
        # print(pp)
        loss = criterion(pred, y_train)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()
        train_l_sum = loss.item()
        if epoch == 1:

            print(
                'epoch %d, loss %.4f'
                % (epoch + 1, train_l_sum))
            break
    # px.line(y=np.array(y_train.cpu().detach_())[:, 0]).show()

def test():
    with torch.no_grad():
        pred = model(x_test)
        loss = criterion(pred, y_test)
        print(loss)
        print(pred[0])
        print(y_test[0])

In [15]:
train(500000)

epoch 2, loss 0.0154


In [16]:
test()

tensor(0.0157, device='cuda:0')
tensor([0.1236, 0.1782, 0.1238, 0.1396, 0.1315, 0.1862, 0.1170],
       device='cuda:0')
tensor([0.0000, 0.0200, 0.1300, 0.3200, 0.3200, 0.1700, 0.0300],
       device='cuda:0')


In [17]:
w = "eerie"
w = [dic[i] for i in w]
[w]

[[4, 4, 17, 8, 4]]

In [18]:
w = embedding(torch.LongTensor([[4, 4, 17, 8, 4]])).view(-1, 5)
w = w.to(device)
w

tensor([[ 0.5624,  0.5624,  0.3245, -0.8780,  0.5624]], device='cuda:0',
       grad_fn=<ToCopyBackward0>)

In [19]:
model(w)

tensor([[0.1341, 0.1455, 0.1652, 0.1399, 0.1440, 0.1485, 0.1227]],
       device='cuda:0', grad_fn=<DivBackward0>)